The main objective of this article is make a model with keras in order to identify whether a text tweet contains toxic comments or not in this particular case of sentiment analysis.

In [ ]:
pip -q install demoji

### Import needed libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import re
from sklearn.metrics import accuracy_score
from nltk.stem.snowball import SnowballStemmer
from sklearn.base import BaseEstimator, TransformerMixin
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from keras.layers import Reshape, merge, Concatenate, Lambda, Average
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.linear_model import SGDClassifier
import logging
from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings;warnings.filterwarnings('ignore')
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Dataset importing and preprocessing

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ar_dataset.csv")
df.shape

(3353, 7)

In [ ]:
df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group
0,1,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual
1,2,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other
2,3,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual
3,4,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other
4,5,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   HITId                3353 non-null   int64 
 1   tweet                3353 non-null   object
 2   sentiment            3353 non-null   object
 3   directness           3353 non-null   object
 4   annotator_sentiment  3353 non-null   object
 5   target               3353 non-null   object
 6   group                3353 non-null   object
dtypes: int64(1), object(6)
memory usage: 183.5+ KB


In [ ]:
df['sentiment'].value_counts()

offensive                                          950
normal                                             915
hateful                                            460
abusive_disrespectful                              205
abusive_hateful                                    168
disrespectful                                      167
disrespectful_normal                               103
abusive_normal                                      81
abusive_offensive                                   77
hateful_disrespectful                               40
offensive_disrespectful                             32
abusive_offensive_hateful_disrespectful_normal      23
offensive_normal                                    21
abusive                                             19
hateful_normal                                      15
fearful                                             12
offensive_hateful                                   11
abusive_offensive_disrespectful_hateful_normal      10
fearful_ab

In [ ]:
df['directness'].value_counts()

indirect    1933
direct      1420
Name: directness, dtype: int64

In [ ]:
df['target'].value_counts()

other         1782
origin         877
gender         548
religion       145
disability       1
Name: target, dtype: int64

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

(3353, 7)

## Data cleaning 





In [ ]:
import demoji
demoji.download_codes()
emojis_ar = {}
for line in df['tweet']:
  emoji_summary = demoji.findall(line)
  if len(emoji_summary)==0:
    continue;
  elif len(emoji_summary)==1:
    emojis_ar.update(emoji_summary)

print(emojis_ar)

... OK (Got response in 0.17 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
{'♥️': 'heart suit', '☺️': 'smiling face', '‼️': 'double exclamation mark', '♥': 'heart suit'}


In [ ]:
stop_words = set(stopwords.words('arabic'))
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text
def remove_numbers(text):
    text = re.sub("0", "", text)
    text = re.sub("1", "", text)
    text = re.sub("2", "", text)
    text = re.sub("3", "", text)
    text = re.sub("4", "", text)
    text = re.sub("5", "", text)
    text = re.sub("6", "", text)
    text = re.sub("7", "", text)
    text = re.sub("8", "", text)
    text = re.sub("9", "", text)
    text = re.sub("٠", "", text)
    text = re.sub("١", "", text)
    text = re.sub("٢", "", text)
    text = re.sub("٣", "", text)
    text = re.sub("٤", "", text)
    text = re.sub("٥", "", text)
    text = re.sub("٦", "", text)
    text = re.sub("٧", "", text)
    text = re.sub("٨", "", text)
    text = re.sub("٩", "", text)
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "♥️"
        "☺️"
        "‼️"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
def remove_tag(text):
    regrex_pattern = re.compile(pattern = 
        u"@user"  # emoticons
        u"@url"  # symbols & pictographs

      
                           )
    text= re.sub(r"user", " ", str(text))
    text= re.sub(r"url", " ", str(text))
    text = re.sub(r'\s*[A-Za-z]+\b', '' , str(text))
    return text


def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text=remove_repeating_char(text)
    text=remove_tag(text)
    text=normalize_arabic(text)
    text=remove_punctuations(text)
    text=remove_numbers(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text
df['cleaned_text'] = df['tweet'].apply(clean_text)
#df['tweet'] = df['tweet'].apply(clean_text)
df['label'] = np.where(df['sentiment']=='normal',0,1)
df.sample(5)

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,cleaned_text,label
1092,1093,خنازير جلدهم تخين فعلا,hateful,indirect,anger_disgust,origin,other,خنازير جلدهم تخين فعلا,1
900,901,من اسباب ظاهرة التحرش بالمرأة: @url,normal,indirect,indifference,other,women,اسباب ظاهره التحرش بالمراه,0
486,487,@user @user هههه اعرف ان الحقيقه دائما مؤلمه و...,hateful,direct,indifference,origin,individual,ه اعرف ان الحقيقه داءما مءلمه وبعدين بول البعي...,1
1575,1576,@user @user هذه بقره ولا خنزير هههههههههههههه,disrespectful,indirect,shock,gender,women,بقره خنزير ه,1
155,156,@user @user انت رجاع خود جرعة من بول البعير ول...,offensive,direct,shock,origin,individual,انت رجاع خود جرعه بول البعير تدخل بيناتنا زق,1


###Training random forest classifier

In [ ]:
import sklearn
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.3, stratify=df['label'])
tfidf = TfidfVectorizer(lowercase=False)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
rf = RandomForestClassifier()
rf_tfidf = rf.fit(X_train_tfidf, y_train)
y_pred = rf_tfidf.predict(X_test_tfidf)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
F1 = sklearn.metrics.f1_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}/F1: {}'.format(round(precision, 3), round(recall, 3), round(accuracy, 3),round(F1,3)))

Precision: 0.78 / Recall: 0.923 / Accuracy: 0.755/F1: 0.846


In [ ]:
import sklearn
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_test, y_pred)
sklearn.metrics.auc(fpr, tpr)

0.616241760975003

In [ ]:
 df.iloc[3350]['tweet']

'الكلب لا يعض اذن اخوه كذابين خنازير @url'

In [ ]:
clean_text(df.iloc[3350]['tweet'])

'الكلب يعض اذن اخوه كذابين خنازير'

### Testing the cleaner functions

Check the original tweets and the cleaned ones to see how good our cleaner is

In [ ]:
for idx in df.tail(40).index:
  print(df.iloc[idx]['tweet'],'\n'  , df.iloc[idx]['cleaned_text'], idx)
  print("************")

@user كلش مو سلعة بايرة 
 كلش مو سلعه بايره 3313
************
@user ما هو تخصص الشنقيذي هذا هل هو مدرس في السياسة ام عام الإجرام ام انه جاء ليقبض بالدولار ويكذب على خنازير ا… @url 
 تخص الشنقيذي مدرس السياسه ام عام الاجرام ام انه جاء ليقبض بالدولار ويكذب علي خنازير ا 3314
************
السيسي: اللاجئون يعيشون حياة كريمة في مصر!! @url 
 السيسي الاجءون يعيشون حياه كريمه مصر 3315
************
@user ما بحاجة ايران تعمل هألشي وما يحصل معهم هو نتيجة شربهم بول البعير .... 
 بحاجه ايران تعمل هالشي يحصل معهم نتيجه شربهم بول البعير 3316
************
@user هههه وانا من جنوب افريقيا انا العربي ولد الإعراب و بعد تعالي عطيني اصلك اذا انتي صادقه عربيه 
 ه وانا جنوب افريقيا انا العربي ولد الاعراب و تعالي عطيني اصلك اذا انتي صادقه عربيه 3317
************
البنات صارو يذبحون يطعنون يدخنون يشيشون ويشربون وانا للحين استحي ادخل على الحريم لحالي . 
 البنات صارو يذبحون يطعنون يدخنون يشيشون ويشربون وانا لحين استحي ادخل علي الحريم لحالي 3318
************
أبناء الإبل يتفاخرون بأن جمالهم المشاركة في السباق الذي يع

## Toxic Models


In [ ]:
encoder = LabelEncoder()
X = df['cleaned_text']
out1 = df['label']
out1 = encoder.fit_transform(out1)
out1 = np_utils.to_categorical(out1)
out2 = df['target']
out2 = encoder.fit_transform(out2)
out2 = np_utils.to_categorical(out2)
out3 = df['directness']
out3 = encoder.fit_transform(out3)
out3 = np_utils.to_categorical(out3)

In [ ]:
print(X.shape)
print(out1.shape)
print(out2.shape)
print(out3.shape)

(3353,)
(3353, 2)
(3353, 5)
(3353, 2)


In [ ]:
'''
# adapted from https://github.com/bakrianoo/aravec
import gensim
import spacy 
import nltk 

# Load AraVec model from gensim    
model = gensim.models.Word2Vec.load("/content/drive/MyDrive/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.mdl")

# export to word2vec format for spacy model
model.wv.save_word2vec_format("/content/drive/MyDrive/full_grams_cbow_300_twitter/aravec.txt")
# run the following command gzip ./spacyModel/aravec.txt   
# run the following command python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz
 
# load spacy model 
nlp = model
'''

'\n# adapted from https://github.com/bakrianoo/aravec\nimport gensim\nimport spacy \nimport nltk \n\n# Load AraVec model from gensim    \nmodel = gensim.models.Word2Vec.load("/content/drive/MyDrive/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.mdl")\n\n# export to word2vec format for spacy model\nmodel.wv.save_word2vec_format("/content/drive/MyDrive/full_grams_cbow_300_twitter/aravec.txt")\n# run the following command gzip ./spacyModel/aravec.txt   \n# run the following command python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz\n \n# load spacy model \nnlp = model\n'

In [ ]:
#!gzip /content/drive/MyDrive/full_grams_cbow_300_twitter/aravec.txt

In [ ]:
#!python -m spacy  init-model ar /content/drive/MyDrive/full_grams_cbow_300_twitter/spacy.aravec.model --vectors-loc /content/drive/MyDrive/full_grams_cbow_300_twitter/aravec.txt.gz

In [ ]:
import spacy
nlp = spacy.load("/content/drive/MyDrive/full_grams_cbow_300_twitter/spacy.aravec.model")

#### Build the embedding

In [ ]:
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(df['cleaned_text'])
embeddings_index = np.zeros((30000+ 1, 300))
for word, idx in tokenizer.word_index.items():
    try:
          embedding = nlp.vocab[word].vector
          embeddings_index[idx] = embedding
    except:
      pass

check the embedding

In [ ]:
embeddings_index[1]

array([ 1.03002965e+00, -3.81823063e-01, -4.37067658e-01,  1.29609096e+00,
        6.70309007e-01, -5.18572450e-01,  2.62831151e-01, -5.04519999e-01,
        6.75375223e-01, -2.50067139e+00,  5.55026650e-01,  8.87341917e-01,
        7.81125665e-01, -1.75152242e+00,  1.03286052e+00, -8.19581926e-01,
        2.08446369e-01, -7.88062274e-01,  1.70702565e+00,  6.98756754e-01,
       -3.52198452e-01, -9.46044549e-02, -1.29057443e+00,  5.76536477e-01,
       -3.65140051e-01,  4.04210597e-01, -5.78187779e-02, -2.29919839e+00,
        3.77723724e-01,  5.65209329e-01, -2.78362006e-01,  1.02985966e+00,
        1.92780411e+00, -2.70805407e+00,  8.56814533e-02, -1.04309380e+00,
       -1.24227226e+00,  1.02873385e-01,  1.94390154e+00, -8.93592060e-01,
       -1.71306443e+00, -1.85619438e+00,  1.61913142e-01, -1.28437376e+00,
        4.76471901e-01,  3.40174168e-01, -7.25198567e-01, -2.96777815e-01,
        2.18076214e-01, -4.23369884e-01,  5.93411140e-02, -3.34327757e-01,
        4.53772023e-02,  

### ToxicModel:Base model

In [ ]:
class KerasTextClassifier(BaseEstimator, TransformerMixin):
  '''Wrapper class for keras text classification models that takes raw text as input.'''
  
  def __init__(self, max_words=30000, input_length=20, emb_dim=300, n_classes=2, epochs=15, batch_size=64, emb_idx=0):
    self.max_words = max_words
    self.input_length = input_length
    self.emb_dim = emb_dim
    self.n_classes = n_classes
    self.epochs = epochs
    self.bs = batch_size
    self.embeddings_index = emb_idx
    self.tokenizer = Tokenizer(num_words=self.max_words+1, lower=True, split=' ')
    self.model = self._get_model()
    return self.model.summary()
    
  def _get_model(self):
    input_text = Input((self.input_length,))
    text_embedding = Embedding(input_dim=self.max_words+1, output_dim=self.emb_dim, input_length=self.input_length, 
                               mask_zero=False, weights=[self.embeddings_index], trainable=False)(input_text)
    text_embedding = SpatialDropout1D(0.4)(text_embedding)
    bilstm =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(text_embedding)
    x = Dropout(0.5)(bilstm)
    x =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(x)
    x = Dropout(0.5)(x)
    x =(LSTM(units=50,  recurrent_dropout=0.2))(x)
    out = Dense(units=2, activation="softmax", name='sentiment')(x)
    model = Model(inputs=[input_text],outputs=[out])
    model.compile(optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])
    return model
  
  def _get_sequences(self, texts):
    seqs = self.tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=self.input_length, value=0)
  
  def _preprocess(self, texts):
    return [clean_text(x) for x in texts]
    # return [x for x in texts]
  
  def fit(self, X, y):
    '''Fit the vocabulary and the model.
       :params: X: list of texts. y: labels.
    '''
    self.tokenizer.fit_on_texts(X)
    self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.max_words}
    self.tokenizer.word_index[self.tokenizer.oov_token] = self.max_words + 1
    seqs = self._get_sequences(self._preprocess(X))
    self.model.fit([seqs ], y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1)
  
  def predict_proba(self, X, y=None):
    seqs = self._get_sequences(self._preprocess(X))
    return self.model.predict(seqs)
  
  def predict(self, X, y=None):
    return np.argmax(self.predict_proba(X), axis=1)
  
  def score(self, X, y):
    y_pred = self.predict(X)
    return accuracy_score(np.argmax(y, axis=1), y_pred)

In [ ]:
text_model = KerasTextClassifier(emb_idx= embeddings_index)

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 20)]              0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 20, 300)           9000300   
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 20, 300)           0         
_________________________________________________________________
lstm_58 (LSTM)               (None, 20, 50)            70200     
_________________________________________________________________
dropout_28 (Dropout)         (None, 20, 50)            0         
_________________________________________________________________
lstm_59 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
dropout_29 (Dropout)         (None, 20, 50)            0  

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(X, out1, out2, out3, test_size = 0.1, random_state = 40, stratify=out1)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y1_train.shape)
print(y1_test.shape)
print(y2_train.shape)
print(y2_test.shape)
print(y3_train.shape)
print(y3_test.shape)

(3017,)
(336,)
(3017, 2)
(336, 2)
(3017, 5)
(336, 5)
(3017, 2)
(336, 2)


#### Train  the model

In [ ]:
text_model.fit(x_train, y1_train)

Epoch 1/15
43/43 [==============================] - 13s 123ms/step - loss: 0.6151 - accuracy: 0.7073 - val_loss: 0.5875 - val_accuracy: 0.7020
Epoch 2/15
43/43 [==============================] - 4s 104ms/step - loss: 0.5529 - accuracy: 0.7395 - val_loss: 0.5764 - val_accuracy: 0.6821
Epoch 3/15
43/43 [==============================] - 4s 102ms/step - loss: 0.5160 - accuracy: 0.7525 - val_loss: 0.5715 - val_accuracy: 0.7119
Epoch 4/15
43/43 [==============================] - 5s 106ms/step - loss: 0.5164 - accuracy: 0.7492 - val_loss: 0.5630 - val_accuracy: 0.7152
Epoch 5/15
43/43 [==============================] - 4s 102ms/step - loss: 0.5048 - accuracy: 0.7580 - val_loss: 0.5717 - val_accuracy: 0.7185
Epoch 6/15
43/43 [==============================] - 4s 102ms/step - loss: 0.4795 - accuracy: 0.7717 - val_loss: 0.5866 - val_accuracy: 0.7185
Epoch 7/15
43/43 [==============================] - 4s 103ms/step - loss: 0.4769 - accuracy: 0.7770 - val_loss: 0.5847 - val_accuracy: 0.7185
Epoch

#### Evaluate the model

In [ ]:
text_model.score(x_test, y1_test)

0.7529761904761905

In [ ]:
y_pred = text_model.predict_proba(x_test, y1_test )

In [ ]:
accuracy_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))

0.7529761904761905

In [ ]:
import sklearn
precision = precision_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
recall = recall_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
F1 = sklearn.metrics.f1_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Precision: {} / Recall: {} / F1score: {}'.format(round(precision, 3), round(recall, 3), round(F1, 3)))

Precision: 0.782 / Recall: 0.914 / F1score: 0.843


In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
sklearn.metrics.auc(fpr, tpr)

0.6200106913756236

#### Explaning Model1 with ELI5

In [ ]:
from IPython.display import display, HTML

In [ ]:
!pip -q install eli5

     |████████████████████████████████| 112kB 7.9MB/s 


####  Let's check some tweets from the test dataset to review how the model is classifying the tweets


In [ ]:
x_test

526            تحليلكم السياسيشبه تحليل بول البعير تشربوه
2602    يالربع ابلشن واحد تكرمون عنه وسخ يرسل رساءل قذ...
2954                            الكيرفي ياريس عشان التحرش
854                    خليك احمد اقتص الحريم دول انا هنام
1228    الافارقه المحترمين مش مشاركين دوري ابطال افريق...
                              ...                        
827                                      مدري شفتي التحرش
2228    رافضي وافتخر لرفض دينكم القذر دين ابن تيميه وا...
626     اشاعات الاغتصابات السجون والاجءين الافارقه الي...
2585                  تقديري ان التحرش واقعنا يقع بالتالي
667                                    الحريم الحين تنخنق
Name: cleaned_text, Length: 336, dtype: object

In [ ]:
import eli5
from eli5.lime import TextExplainer

for idx in x_test.index[190:210]:
  te = TextExplainer(random_state=42)
  te.fit(clean_text(x_test[idx]), text_model.predict_proba, )
  #te.fit(x_test[idx], text_model.predict_proba, )
  print("Real Class:",  ["Non Toxic" if x == 0 else "Toxic" for x in [df.iloc[idx]['label']]])
  print("Text uncleaned tweet:", df.iloc[idx]['tweet'])
  print("ELI5 Predicted Class:")
  HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))
 

Real Class: ['Non Toxic']
Text uncleaned tweet: المغازل يعتبر تحرش؟ تتطبق عليه عقوبة التحرش يعني؟
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: لم تكوني بتتونسي مع صحباتك وهم متزوجات وانتي بايره  @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user ماهو مبرر ياكثر الحريم الباردات بالجنس ومع ذالك كايخون زوجته + كفوكم سواق
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: وانت لا تتهاوش معى وتحط عقلك بعقل مره خلاص مشها وخلها قبلك الله يعنكم على الحريم بس
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user @user روحي يابنت البعير انتي شابعه من بول البعير وجايه تسوي نفسك كيوته وراقي… @user
ELI5 Predicted Class:


...

Real Class: ['Toxic']
Text uncleaned tweet: @user @user الحصالة اللي ع شگل خنزير لونها وردي
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: رعاة الغنم دول،  شاربي بول البعير
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: @user وجه نظر.. اذا كان يمنع النقاب فنتمني عدم العرى من بعض البنات  حتي نقضي ع التحرش اهو يبقي ضربنا عصفورين @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: مؤمن زكريا ده وسخ لدرجة انه يوم ما جاب جول جابه في فريق استاذة لارا فقيه @url
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: ذكرت كلام تجمع قبل بمعنى كلامهم الحريم ماعندهن هياط بالكلمات وفعلاً تو قريتها حسّيت اللي كاتبه دكتور او دكتوره
ELI5 Predicted Class:


### Toxic Model2:Multitask with 2 outputs

In [ ]:
class KerasTextClassifier2(BaseEstimator, TransformerMixin):
  '''Wrapper class for keras text classification models that takes raw text as input.'''
  
  def __init__(self, max_words=30000, input_length=20, emb_dim=300, n_classes=2, epochs=30, batch_size=64, emb_idx=0):
    self.max_words = max_words
    self.input_length = input_length
    self.emb_dim = emb_dim
    self.n_classes = n_classes
    self.epochs = epochs
    self.bs = batch_size
    self.embeddings_index = emb_idx
    self.tokenizer = Tokenizer(num_words=self.max_words+1, lower=True, split=' ')
    self.model = self._get_model()
    return self.model.summary()
    
  def _get_model(self):
    input_text = Input((self.input_length,))
    text_embedding = Embedding(input_dim=self.max_words+1, output_dim=self.emb_dim, input_length=self.input_length, 
                               mask_zero=False, weights=[self.embeddings_index], trainable=False)(input_text)
    text_embedding = SpatialDropout1D(0.4)(text_embedding)
    bilstm =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(text_embedding)
    x = Dropout(0.5)(bilstm)
    x =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(x)
    x = Dropout(0.5)(x)
    out1 =(LSTM(units=50,  recurrent_dropout=0.2))(x)
    out2 =(LSTM(units=50,  recurrent_dropout=0.2))(x)
    out1 = Dense(units=2, activation="softmax", name='sentiment')(out1)
    out2 = Dense(units=5, activation="softmax", name='target')(out2)
    model = Model(inputs=[input_text],outputs=[out1,out2])
    model.compile(optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])
    return model
  
  def _get_sequences(self, texts):
    seqs = self.tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=self.input_length, value=0)
  
  def _preprocess(self, texts):
    return [clean_text(x) for x in texts]
    # return [x for x in texts]
  
  def fit(self, X, y):
    '''Fit the vocabulary and the model.
       :params: X: list of texts. y: labels.
    '''
    self.tokenizer.fit_on_texts(X)
    self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.max_words}
    self.tokenizer.word_index[self.tokenizer.oov_token] = self.max_words + 1
    seqs = self._get_sequences(self._preprocess(X))
    self.model.fit([seqs ], y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1)
  
  def predict_proba(self, X, y=None):
    seqs = self._get_sequences(self._preprocess(X))
    return self.model.predict(seqs)
  def predict_proba2(self, X, y=None):
    seqs = self._get_sequences(self._preprocess(X))
    return self.model.predict(seqs)[0]
  
  def predict(self, X, y=None):
    return np.argmax(self.predict_proba(X), axis=1)
  
  def score(self, X, y):
    y_pred = self.predict(X)
    return accuracy_score(np.argmax(y, axis=1), y_pred)

#### Train the model

In [ ]:
text_model_multitask_2 = KerasTextClassifier2(emb_idx= embeddings_index)

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 20, 300)      9000300     input_12[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_11 (SpatialDr (None, 20, 300)      0           embedding_11[0][0]               
__________________________________________________________________________________________________
lstm_44 (LSTM)                  (None, 20, 50)       70200       spatial_dropout1d_11[0][0]       
___________________________________________________________________________________________

In [ ]:
text_model_multitask_2.fit(x_train, [y1_train, y2_train] )

Epoch 1/30
43/43 [==============================] - 16s 155ms/step - loss: 2.0232 - sentiment_loss: 0.6185 - target_loss: 1.4048 - sentiment_accuracy: 0.7074 - target_accuracy: 0.4639 - val_loss: 1.7142 - val_sentiment_loss: 0.6211 - val_target_loss: 1.0931 - val_sentiment_accuracy: 0.7020 - val_target_accuracy: 0.5397
Epoch 2/30
43/43 [==============================] - 5s 121ms/step - loss: 1.6725 - sentiment_loss: 0.5566 - target_loss: 1.1159 - sentiment_accuracy: 0.7418 - target_accuracy: 0.5302 - val_loss: 1.5861 - val_sentiment_loss: 0.5800 - val_target_loss: 1.0062 - val_sentiment_accuracy: 0.6887 - val_target_accuracy: 0.5430
Epoch 3/30
43/43 [==============================] - 5s 123ms/step - loss: 1.5772 - sentiment_loss: 0.5507 - target_loss: 1.0265 - sentiment_accuracy: 0.7143 - target_accuracy: 0.5416 - val_loss: 1.5788 - val_sentiment_loss: 0.5992 - val_target_loss: 0.9796 - val_sentiment_accuracy: 0.7020 - val_target_accuracy: 0.5298
Epoch 4/30
43/43 [=====================

#### Evaluate the model

In [ ]:
preds = text_model_multitask_2.predict_proba(x_test)

In [ ]:
pred1=[]
pred2=[]
for i in range(len(preds[0])):
    pred1.append(preds[0][i])
    pred2.append(preds[1][i])
pred1=np.array(pred1)
pred2=np.array(pred2)

In [ ]:
sentiment_accuracy = accuracy_score(y1_test.argmax(axis=1), pred1.argmax(axis=1))
target_accuracy = accuracy_score(y2_test.argmax(axis=1), pred2.argmax(axis=1))

print('sentiment_accuracy = ', sentiment_accuracy, ', target_accuracy = ', target_accuracy)

sentiment_accuracy =  0.7619047619047619 , target_accuracy =  0.5178571428571429


In [ ]:
precision = precision_score(y1_test.argmax(axis=1), pred1.argmax(axis=1))
recall = recall_score(y1_test.argmax(axis=1), pred1.argmax(axis=1))
F1 = sklearn.metrics.f1_score(y1_test.argmax(axis=1), pred1.argmax(axis=1))
print('Precision: {} / Recall: {} / F1score: {}'.format(round(precision, 3), round(recall, 3), round(F1, 3)))

Precision: 0.787 / Recall: 0.922 / F1score: 0.849


In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y1_test.argmax(axis=1), pred1.argmax(axis=1))
sklearn.metrics.auc(fpr, tpr)

0.629543834640057

#### Explaning Model2 with ELI5

In [ ]:
import eli5
from eli5.lime import TextExplainer
import sklearn

te = TextExplainer(random_state=42)
te.fit(clean_text(x_test), text_model_multitask_2.predict_proba2,)

TextExplainer(char_based=False,
              clf=SGDClassifier(alpha=0.001, average=False, class_weight=None,
                                early_stopping=False, epsilon=0.1, eta0=0.0,
                                fit_intercept=True, l1_ratio=0.15,
                                learning_rate='optimal', loss='log',
                                max_iter=1000, n_iter_no_change=5, n_jobs=None,
                                penalty='elasticnet', power_t=0.5,
                                random_state=RandomState(MT19937) at 0x7FA22261A490,
                                shuffle=True, tol=0....
                                          weights=array([0.7, 0.3])),
              token_pattern='(?u)\\b\\w+\\b',
              vec=CountVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.int64'>, encoding='utf-8',
                                  input='content', lowercase=True, ma

In [ ]:
import eli5
from eli5.lime import TextExplainer

for idx in x_test.index[190:210]:
  te = TextExplainer(random_state=42)
  te.fit(clean_text(x_test[idx]),text_model_multitask_2.predict_proba2, )
  #te.fit(x_test[idx], text_model.predict_proba, )
  print("Real Class:",  ["Non Toxic" if x == 0 else "Toxic" for x in [df.iloc[idx]['label']]])
  print("Text uncleaned tweet:", df.iloc[idx]['tweet'])
  print("ELI5 Predicted Class:")
  HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))
 

Real Class: ['Non Toxic']
Text uncleaned tweet: المغازل يعتبر تحرش؟ تتطبق عليه عقوبة التحرش يعني؟
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: لم تكوني بتتونسي مع صحباتك وهم متزوجات وانتي بايره  @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user ماهو مبرر ياكثر الحريم الباردات بالجنس ومع ذالك كايخون زوجته + كفوكم سواق
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: وانت لا تتهاوش معى وتحط عقلك بعقل مره خلاص مشها وخلها قبلك الله يعنكم على الحريم بس
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user @user روحي يابنت البعير انتي شابعه من بول البعير وجايه تسوي نفسك كيوته وراقي… @user
ELI5 Predicted Class:


...

Real Class: ['Toxic']
Text uncleaned tweet: @user @user الحصالة اللي ع شگل خنزير لونها وردي
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: رعاة الغنم دول،  شاربي بول البعير
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: @user وجه نظر.. اذا كان يمنع النقاب فنتمني عدم العرى من بعض البنات  حتي نقضي ع التحرش اهو يبقي ضربنا عصفورين @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: مؤمن زكريا ده وسخ لدرجة انه يوم ما جاب جول جابه في فريق استاذة لارا فقيه @url
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: ذكرت كلام تجمع قبل بمعنى كلامهم الحريم ماعندهن هياط بالكلمات وفعلاً تو قريتها حسّيت اللي كاتبه دكتور او دكتوره
ELI5 Predicted Class:


In [ ]:
HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))

<IPython.core.display.HTML object>

In [ ]:
import eli5
from eli5.lime import TextExplainer

for idx in x_test.index[190:210]:
  #print("Real Class:",  ["Non Toxic" if x == 0 else "Toxic" for x in [df.iloc[idx]['label']]])
  print("Text uncleaned tweet:", df.iloc[idx]['tweet'])
  print("ELI5 Predicted Class:")
  HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))

Text uncleaned tweet: المغازل يعتبر تحرش؟ تتطبق عليه عقوبة التحرش يعني؟
ELI5 Predicted Class:


Text uncleaned tweet: لم تكوني بتتونسي مع صحباتك وهم متزوجات وانتي بايره  @url
ELI5 Predicted Class:


Text uncleaned tweet: @user @user ماهو مبرر ياكثر الحريم الباردات بالجنس ومع ذالك كايخون زوجته + كفوكم سواق
ELI5 Predicted Class:


Text uncleaned tweet: وانت لا تتهاوش معى وتحط عقلك بعقل مره خلاص مشها وخلها قبلك الله يعنكم على الحريم بس
ELI5 Predicted Class:


Text uncleaned tweet: @user @user @user روحي يابنت البعير انتي شابعه من بول البعير وجايه تسوي نفسك كيوته وراقي… @user
ELI5 Predicted Class:


...

Text uncleaned tweet: @user @user الحصالة اللي ع شگل خنزير لونها وردي
ELI5 Predicted Class:


Text uncleaned tweet: رعاة الغنم دول،  شاربي بول البعير
ELI5 Predicted Class:


Text uncleaned tweet: @user وجه نظر.. اذا كان يمنع النقاب فنتمني عدم العرى من بعض البنات  حتي نقضي ع التحرش اهو يبقي ضربنا عصفورين @url
ELI5 Predicted Class:


Text uncleaned tweet: مؤمن زكريا ده وسخ لدرجة انه يوم ما جاب جول جابه في فريق استاذة لارا فقيه @url
ELI5 Predicted Class:


Text uncleaned tweet: ذكرت كلام تجمع قبل بمعنى كلامهم الحريم ماعندهن هياط بالكلمات وفعلاً تو قريتها حسّيت اللي كاتبه دكتور او دكتوره
ELI5 Predicted Class:


### Toxic Model3:Multitask with 3 outputs

In [ ]:
class KerasTextClassifier3(BaseEstimator, TransformerMixin):
  '''Wrapper class for keras text classification models that takes raw text as input.'''
  
  def __init__(self, max_words=30000, input_length=20, emb_dim=300, n_classes=2, epochs=30, batch_size=64, emb_idx=0):
    self.max_words = max_words
    self.input_length = input_length
    self.emb_dim = emb_dim
    self.n_classes = n_classes
    self.epochs = epochs
    self.bs = batch_size
    self.embeddings_index = emb_idx
    self.tokenizer = Tokenizer(num_words=self.max_words+1, lower=True, split=' ')
    self.model = self._get_model()
    return self.model.summary()
    
  def _get_model(self):
    input_text = Input((self.input_length,))
    text_embedding = Embedding(input_dim=self.max_words+1, output_dim=self.emb_dim, input_length=self.input_length, 
                               mask_zero=False, weights=[self.embeddings_index], trainable=False)(input_text)
    text_embedding = SpatialDropout1D(0.4)(text_embedding)
    bilstm =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(text_embedding)
    x = Dropout(0.5)(bilstm)
    x =(LSTM(units=50,  recurrent_dropout=0.2, return_sequences = True))(x)
    x = Dropout(0.5)(x)
    out1 =(LSTM(units=50,  recurrent_dropout=0.2))(x)
    out2 =(LSTM(units=50,  recurrent_dropout=0.2))(x)
    out3 =(LSTM(units=50,  recurrent_dropout=0.2))(x)

    out1 = Dense(units=2, activation="softmax", name='sentiment')(out1)
    out2 = Dense(units=5, activation="softmax", name='target')(out2)
    out3 = Dense(units=2, activation="softmax", name='directness')(out3)

    model = Model(inputs=[input_text],outputs=[out1,out2,out3])

    model.compile(
     loss="categorical_crossentropy",
     optimizer='adam',
     metrics=['accuracy']
    )
    return model
  
  def _get_sequences(self, texts):
    seqs = self.tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=self.input_length, value=0)
  
  def _preprocess(self, texts):
    return [clean_text(x) for x in texts]
    # return [x for x in texts]
  
  def fit(self, X, y):
    '''Fit the vocabulary and the model.
       :params: X: list of texts. y: labels.
    '''
    self.tokenizer.fit_on_texts(X)
    self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.max_words}
    self.tokenizer.word_index[self.tokenizer.oov_token] = self.max_words + 1
    seqs = self._get_sequences(self._preprocess(X))
    self.model.fit([seqs ], y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1)
    self.model.save('modelx.hdf5')
  
  def predict_proba(self, X, y=None):
    seqs = self._get_sequences(self._preprocess(X))
    #self.model.predict(seqs)
    return self.model.predict(seqs)[0:335][0]
  
  def predict(self, X, y=None):
    return np.argmax(self.predict_proba(X), axis=1)
  
  def score(self, X, y ):
    y_pred = self.predict(X)
    return accuracy_score(np.argmax(y[1,:], axis=1), y_pred[1,:])

#### Train the model

In [ ]:
text_model_multitask_3 = KerasTextClassifier3(emb_idx= embeddings_index)

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 20, 300)      9000300     input_14[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_13 (SpatialDr (None, 20, 300)      0           embedding_13[0][0]               
__________________________________________________________________________________________________
lstm_53 (LSTM)                  (None, 20, 50)       70200       spatial_dropout1d_13[0][0]       
___________________________________________________________________________________________

In [ ]:
text_model_multitask_3.fit(x_train, [y1_train, y2_train, y3_train] )

Epoch 1/30
43/43 [==============================] - 21s 188ms/step - loss: 2.7152 - sentiment_loss: 0.6339 - target_loss: 1.3907 - directness_loss: 0.6905 - sentiment_accuracy: 0.6854 - target_accuracy: 0.4634 - directness_accuracy: 0.5374 - val_loss: 2.3890 - val_sentiment_loss: 0.6015 - val_target_loss: 1.0972 - val_directness_loss: 0.6904 - val_sentiment_accuracy: 0.7020 - val_target_accuracy: 0.5397 - val_directness_accuracy: 0.5397
Epoch 2/30
43/43 [==============================] - 6s 144ms/step - loss: 2.3709 - sentiment_loss: 0.5754 - target_loss: 1.1142 - directness_loss: 0.6813 - sentiment_accuracy: 0.7280 - target_accuracy: 0.5278 - directness_accuracy: 0.5672 - val_loss: 2.2816 - val_sentiment_loss: 0.5796 - val_target_loss: 1.0190 - val_directness_loss: 0.6830 - val_sentiment_accuracy: 0.6987 - val_target_accuracy: 0.5199 - val_directness_accuracy: 0.5596
Epoch 3/30
43/43 [==============================] - 6s 147ms/step - loss: 2.2442 - sentiment_loss: 0.5312 - target_loss

#### Evaluate the model

In [ ]:
preds = text_model_multitask_3.predict_proba(x_test, [y1_test, y2_test,y3_test])

In [ ]:
def get_sequences(texts):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=20, value=0)

In [ ]:
xx=get_sequences(x_test)

In [ ]:
from keras.models import load_model
m= load_model('modelx.hdf5')

In [ ]:
y_pred = m.predict(xx)[0:335][0]

In [ ]:
accuracy_score(np.argmax(y1_test, axis=1), np.argmax(y_pred, axis=1))

0.7261904761904762

In [ ]:
y=text_model_multitask_3.predict_proba(x_test)

In [ ]:
precision = precision_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
recall = recall_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
F1 = sklearn.metrics.f1_score(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Precision: {} / Recall: {} / F1score: {}'.format(round(precision, 3), round(recall, 3), round(F1, 3)))

Precision: 0.804 / Recall: 0.824 / F1score: 0.814


In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y1_test.argmax(axis=1), y_pred.argmax(axis=1))
sklearn.metrics.auc(fpr, tpr)

0.6455808980755524

#### Explaning Model3 with ELI5

In [ ]:
import eli5
from eli5.lime import TextExplainer
import sklearn

te = TextExplainer(random_state=42)
te.fit(clean_text(x_test), text_model_multitask_3.predict_proba,)

TextExplainer(char_based=False,
              clf=SGDClassifier(alpha=0.001, average=False, class_weight=None,
                                early_stopping=False, epsilon=0.1, eta0=0.0,
                                fit_intercept=True, l1_ratio=0.15,
                                learning_rate='optimal', loss='log',
                                max_iter=1000, n_iter_no_change=5, n_jobs=None,
                                penalty='elasticnet', power_t=0.5,
                                random_state=RandomState(MT19937) at 0x7F09D62EA5A0,
                                shuffle=True, tol=0....
                                          weights=array([0.7, 0.3])),
              token_pattern='(?u)\\b\\w+\\b',
              vec=CountVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.int64'>, encoding='utf-8',
                                  input='content', lowercase=True, ma

In [ ]:
import eli5
from eli5.lime import TextExplainer

for idx in x_test.index[190:210]:
  te = TextExplainer(random_state=42)
  te.fit(clean_text(x_test[idx]),text_model_multitask_3.predict_proba, )
  #te.fit(x_test[idx], text_model.predict_proba, )

In [ ]:
HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))

<IPython.core.display.HTML object>

In [ ]:
import eli5
from eli5.lime import TextExplainer

for idx in x_test.index[190:210]:
  te = TextExplainer(random_state=42)
  te.fit(clean_text(x_test[idx]),text_model_multitask_3.predict_proba, )
  #te.fit(x_test[idx], text_model.predict_proba, )
  print("Real Class:",  ["Non Toxic" if x == 0 else "Toxic" for x in [df.iloc[idx]['label']]])
  print("Text uncleaned tweet:", df.iloc[idx]['tweet'])
  print("ELI5 Predicted Class:")
  HTML(display((te.show_prediction(target_names=[ 'Non Toxic','Toxic',]))))
 

Real Class: ['Non Toxic']
Text uncleaned tweet: المغازل يعتبر تحرش؟ تتطبق عليه عقوبة التحرش يعني؟
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: لم تكوني بتتونسي مع صحباتك وهم متزوجات وانتي بايره  @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user ماهو مبرر ياكثر الحريم الباردات بالجنس ومع ذالك كايخون زوجته + كفوكم سواق
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: وانت لا تتهاوش معى وتحط عقلك بعقل مره خلاص مشها وخلها قبلك الله يعنكم على الحريم بس
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: @user @user @user روحي يابنت البعير انتي شابعه من بول البعير وجايه تسوي نفسك كيوته وراقي… @user
ELI5 Predicted Class:


...

Real Class: ['Toxic']
Text uncleaned tweet: @user @user الحصالة اللي ع شگل خنزير لونها وردي
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: رعاة الغنم دول،  شاربي بول البعير
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: @user وجه نظر.. اذا كان يمنع النقاب فنتمني عدم العرى من بعض البنات  حتي نقضي ع التحرش اهو يبقي ضربنا عصفورين @url
ELI5 Predicted Class:


Real Class: ['Toxic']
Text uncleaned tweet: مؤمن زكريا ده وسخ لدرجة انه يوم ما جاب جول جابه في فريق استاذة لارا فقيه @url
ELI5 Predicted Class:


Real Class: ['Non Toxic']
Text uncleaned tweet: ذكرت كلام تجمع قبل بمعنى كلامهم الحريم ماعندهن هياط بالكلمات وفعلاً تو قريتها حسّيت اللي كاتبه دكتور او دكتوره
ELI5 Predicted Class:


###References:
Some parts of the code were taken from the following refernces:
1-https://github.com/motazsaad/process-arabic-text
2-https://github.com/bakrianoo/aravec
3-https://www.coursera.org/projects/multi-task-models-keras
4-https://towardsdatascience.com/analyzing-twitter-spheres-through-nlp-techniques-748b0df10b6c